In [1]:
import gym
import random
import numpy as np
from keras.models     import Sequential
from keras.layers     import Dense
from keras.optimizers import Adam

In [3]:
env = gym.make('CartPole-v1')
env.reset()
goal_steps = 500
score_requirement = 60
intial_games = 10000

In [4]:
def play_a_random_game_first():
    for step_index in range(goal_steps):
#         env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print("Step {}:".format(step_index))
        print("action: {}".format(action))
        print("observation: {}".format(observation))
        print("reward: {}".format(reward))
        print("done: {}".format(done))
#         print("info: {}".format(info))
        print("\n")
        if done:
            break
    env.reset()

In [5]:
play_a_random_game_first()

Step 0:
action: 0
observation: [ 0.01143805 -0.2303175  -0.04249558  0.25359435]
reward: 1.0
done: False


Step 1:
action: 1
observation: [ 0.0068317  -0.03461534 -0.03742369 -0.0521838 ]
reward: 1.0
done: False


Step 2:
action: 1
observation: [ 0.0061394   0.16102268 -0.03846737 -0.35643536]
reward: 1.0
done: False


Step 3:
action: 0
observation: [ 0.00935985 -0.03353185 -0.04559607 -0.07612607]
reward: 1.0
done: False


Step 4:
action: 0
observation: [ 0.00868921 -0.22797149 -0.0471186   0.20182966]
reward: 1.0
done: False


Step 5:
action: 1
observation: [ 0.00412978 -0.03220844 -0.043082   -0.10533675]
reward: 1.0
done: False


Step 6:
action: 0
observation: [ 0.00348561 -0.22668737 -0.04518874  0.17344877]
reward: 1.0
done: False


Step 7:
action: 1
observation: [-0.00104813 -0.03094878 -0.04171976 -0.13314036]
reward: 1.0
done: False


Step 8:
action: 0
observation: [-0.00166711 -0.22544907 -0.04438257  0.14609416]
reward: 1.0
done: False


Step 9:
action: 1
observation: [-0.00

In [5]:
def model_data_preparation():
    training_data = []
    accepted_scores = []
    for game_index in range(intial_games):
        score = 0
        game_memory = []
        previous_observation = []
        for step_index in range(goal_steps):
            action = random.randrange(0, 2)
            observation, reward, done, info = env.step(action)
            
            if len(previous_observation) > 0:
                game_memory.append([previous_observation, action])
                
            previous_observation = observation
            score += reward
            if done:
                break
            
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                elif data[1] == 0:
                    output = [1, 0]
                training_data.append([data[0], output])
        
        env.reset()

    print(accepted_scores)
    
    return training_data

In [6]:
training_data = model_data_preparation()

[71.0, 64.0, 62.0, 76.0, 73.0, 89.0, 64.0, 75.0, 65.0, 63.0, 78.0, 71.0, 63.0, 71.0, 62.0, 81.0, 73.0, 62.0, 62.0, 83.0, 83.0, 64.0, 69.0, 60.0, 80.0, 70.0, 68.0, 63.0, 66.0, 80.0, 91.0, 77.0, 62.0, 62.0, 73.0, 66.0, 72.0, 65.0, 61.0, 61.0, 75.0, 66.0, 63.0, 66.0, 65.0, 77.0, 63.0, 60.0, 62.0, 86.0, 68.0, 69.0, 89.0, 60.0, 95.0, 72.0, 102.0, 74.0, 61.0, 61.0, 74.0, 96.0, 62.0, 61.0, 65.0, 74.0, 66.0, 60.0, 61.0, 85.0, 72.0, 66.0, 84.0, 88.0, 62.0, 75.0, 64.0, 67.0, 66.0, 73.0, 82.0, 89.0, 81.0, 103.0, 62.0, 62.0, 73.0, 64.0, 62.0, 103.0, 73.0, 69.0, 62.0, 66.0, 65.0, 61.0, 65.0, 62.0, 64.0, 63.0, 66.0, 60.0, 75.0, 67.0, 61.0, 67.0, 65.0, 62.0, 71.0, 82.0, 107.0, 78.0, 62.0, 67.0, 68.0, 68.0, 62.0, 60.0, 76.0, 68.0, 67.0, 70.0, 64.0, 68.0, 106.0, 68.0, 145.0, 63.0, 65.0, 66.0, 66.0, 124.0, 66.0, 68.0, 73.0, 76.0, 70.0, 69.0, 61.0, 61.0, 70.0, 68.0, 72.0, 67.0, 61.0, 62.0, 66.0, 60.0, 73.0, 83.0, 64.0, 77.0, 66.0, 73.0, 94.0, 62.0]


In [7]:
def build_model(input_size, output_size):
    model = Sequential()
    model.add(Dense(128, input_dim=input_size, activation='relu'))
    model.add(Dense(52, activation='relu'))
    model.add(Dense(output_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())

    return model

In [8]:
def train_model(training_data):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    
    model.fit(X, y, epochs=10)
    return model

NameError: name 'X' is not defined

In [9]:
trained_model = train_model(training_data)

Epoch 1/10
343/343 [==============================] - 0s 869us/step - loss: 0.2505
Epoch 2/10
343/343 [==============================] - 0s 877us/step - loss: 0.2357
Epoch 3/10
343/343 [==============================] - 0s 909us/step - loss: 0.2336
Epoch 4/10
343/343 [==============================] - 0s 975us/step - loss: 0.2335
Epoch 5/10
343/343 [==============================] - 0s 874us/step - loss: 0.2326
Epoch 6/10
343/343 [==============================] - 0s 886us/step - loss: 0.2327
Epoch 7/10
343/343 [==============================] - 0s 880us/step - loss: 0.2325
Epoch 8/10
343/343 [==============================] - 0s 871us/step - loss: 0.2318
Epoch 9/10
343/343 [==============================] - 0s 888us/step - loss: 0.2318
Epoch 10/10
343/343 [==============================] - 0s 898us/step - loss: 0.2313


In [10]:
scores = []
choices = []
for each_game in range(100):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        # Uncomment below line if you want to see how our bot is playing the game.
        env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1, len(prev_obs)))[0])
        
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score+=reward
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))

KeyboardInterrupt: 

In [11]:
env

<TimeLimit<CartPoleEnv<CartPole-v1>>>